In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from PIL import Image
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab.patches import cv2_imshow

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Function to download and display an image
def display_image(url):
    img = Image.open(requests.get(url, stream=True).raw)
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    return img


## Section 1: Language Processing
Application: Sentiment Analysis

In [ ]:
# Load a pre-trained sentiment analysis model from TensorFlow Hub
sentiment_model = hub.load("https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1")

In [ ]:
# Example text data
texts = [
    "I love sunny days!",
    "I hate when it rains all day.",
    "The weather is just perfect.",
    "I'm feeling quite sad today."
]

In [ ]:
# Convert text data to embeddings
text_embeddings = sentiment_model(texts)

In [ ]:
# Function to calculate sentiment score
# Function to calculate sentiment score
def sentiment_score(text_embedding):
    positive_embedding = sentiment_model(["I am happy"])[0]
    negative_embedding = sentiment_model(["I am sad"])[0]
    positive_similarity = np.inner(text_embedding, positive_embedding)
    negative_similarity = np.inner(text_embedding, negative_embedding)
    return positive_similarity - negative_similarity

In [ ]:
# Calculate sentiment scores for each text
scores = [sentiment_score(embed) for embed in text_embeddings]

In [ ]:
# Display the texts and their sentiment scores
for text, score in zip(texts, scores):
    sentiment = "Positive" if score > 0 else "Negative"
    print(f"Text: {text}\nSentiment Score: {score} ({sentiment})\n")

## Section 2: Computer Vision
Application: Object Detection

In [ ]:
# Load a pre-trained object detection model from TensorFlow Hub
detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

# Example image URL
image_url = "https://images.unsplash.com/photo-1593642532973-d31b6557fa68"

In [ ]:
# Display the image
img = display_image(image_url)

In [ ]:
# Convert the image to a tensor
img_tensor = tf.image.convert_image_dtype(np.array(img), tf.float32)[tf.newaxis, ...]

In [ ]:
# Run the object detection model
result = detector(img_tensor)

In [ ]:
# Function to draw bounding boxes on the image
def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.5):
    colors = list(plt.cm.tab20(np.linspace(0, 1, 20)))
    font = cv2.FONT_HERSHEY_SIMPLEX
    for i in range(min(max_boxes, boxes.shape[0])):
        if scores[i] >= min_score:
            color = colors[int(class_names[i]) % len(colors)]
            ymin, xmin, ymax, xmax = tuple(boxes[i])
            image = cv2.rectangle(image, (int(xmin * img.width), int(ymin * img.height)),
                                  (int(xmax * img.width), int(ymax * img.height)), color, 2)
            image = cv2.putText(image, str(int(class_names[i])), (int(xmin * img.width), int(ymin * img.height) - 10),
                                font, 0.5, color, 2)
    return image

In [ ]:
# Extract detection results
boxes = result["detection_boxes"][0].numpy()
class_names = result["detection_classes"][0].numpy()
scores = result["detection_scores"][0].numpy()

In [ ]:
# Draw bounding boxes on the image
output_img = draw_boxes(np.array(img), boxes, class_names, scores)

In [ ]:
# Display the image with bounding boxes
cv2_imshow(output_img)

## Section 3: Natural Language Processing
Application: Text Similarity

In [ ]:
# Example text data
text1 = "AI is transforming the world."
text2 = "Artificial intelligence is changing everything."

In [ ]:
# Convert text data to embeddings
embedding1 = sentiment_model([text1])[0]
embedding2 = sentiment_model([text2])[0]

In [ ]:
# Calculate cosine similarity between the two texts
similarity = cosine_similarity([embedding1], [embedding2])[0][0]

In [ ]:
# Display the texts and their similarity score
print(f"Text 1: {text1}\nText 2: {text2}\nSimilarity Score: {similarity}")